In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
import math
import warnings
from matplotlib import rcParams

import tensorflow as tf
from tensorflow.python.keras.models import Sequential
from tensorflow import keras
from tensorflow.python.keras.layers import Dropout, Dense, LSTM,Activation,GRU,SimpleRNN
from tensorflow.python.keras import backend as K

from keras import regularizers
from keras.losses import mean_squared_error

warnings.filterwarnings("ignore")

In [ ]:
data_init = pd.read_excel("shujuji.xlsx")

x,y = data_init.iloc[:,1:6],data_init.iloc[:,-1]

In [ ]:
# 数据归一化
scaler_input = MinMaxScaler(feature_range=(0, 1))

scaler_output = MinMaxScaler(feature_range=(0, 1))

data_input = data_init.iloc[:,1:-1]
data_output = np.array(data_init.iloc[:,-1]).reshape(-1, 1)


x_data = scaler_input.fit_transform(data_input) # 数据归一化[0,1]
y_data = scaler_output.fit_transform(data_output) # 数据归一化[0,1]




In [ ]:
time_steps = 4

In [ ]:
xx_data = []
yy_data = []

# 测试集：csv表格中前187条数据
# 利用for循环，遍历整个训练集，提取训练集中连续5条数据作为输入特征x_train，第6条的数据作为标签
for i in range(time_steps, len(x_data)):
    xx_data.append(x_data[i - time_steps:i, :].tolist())
    yy_data.append(y_data[i])
    
yy_data = np.array(yy_data)

In [ ]:
## SVR
xxx_data = []
for data in xx_data:
    model_data = []
    # 分离子浓度计算，共5个
    for k in range(5):
        model_data_include = 0
        for j in range(time_steps):
            # 距离越近，权重越高，分别是0.1-0.2-0.3-0.4
            model_data_include += 0.1*(j+1)*data[j][k]
        model_data.append(model_data_include)
    xxx_data.append(model_data)
    

In [ ]:

rmse = 0

train_X, train_y = np.array(xxx_data)[:184], yy_data[:184]
test_X, test_y = np.array(xxx_data)[184:], yy_data[184:]

#LSTM/GRU
train_X_nn, train_y_nn = np.array(xx_data)[:184,:], yy_data[:184]
test_X_nn, test_y_nn = np.array(xx_data)[184:,:], yy_data[184:]


In [ ]:
x_train,y_train = np.array(train_X_nn),np.array(train_y_nn)

In [ ]:
x_train.shape,y_train.shape

In [ ]:
x_test,y_test = np.array(test_X_nn),np.array(test_y_nn)
x_test.shape,y_test.shape

In [ ]:
def show_data(test_y,predict_y,title):
    plt.figure(figsize=(10,8),dpi=100)
    x = [i for i in range(test_y.shape[0])]
    plt.plot(x, test_y, color='navy',lw = 2, label='Actual data')
    plt.plot(x, predict_y, color='c',lw = 2, label='Predict data')
    x_label = [i for i in x[::5]]
    
    y_label = [j for j in range(0,500,50)]
    plt.yticks(y_label)
    plt.xticks(x_label)
    plt.xlabel('Time/d')
    plt.ylabel('COD')
    plt.title(title)
    plt.legend()
    plt.show()

In [ ]:
model_BP = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(x_train.shape[1], x_train.shape[2])),
    tf.keras.layers.Dense(60, activation = 'relu'),
    tf.keras.layers.Dense(120, activation = 'relu'),
    tf.keras.layers.Dense(240, activation = 'relu'),
    tf.keras.layers.Dense(480, activation = 'relu'),
    tf.keras.layers.Dense(240, activation = 'relu'),
    tf.keras.layers.Dense(120, activation = 'relu'),
    tf.keras.layers.Dense(60, activation = 'relu'),
    tf.keras.layers.Dense(1)
])

model_BP.compile(optimizer = 'adam', loss = 'mse')
callbacks = [keras.callbacks.EarlyStopping(patience = 5, min_delta = 1e-3)]

#训练
history_BP = model_BP.fit(x_train, y_train, batch_size=128, epochs=200, validation_data=(x_test, y_test), validation_freq=1)


model_BP.summary() #打印

In [ ]:
y_test1 = scaler_output.inverse_transform(y_test)

In [ ]:
y_predict_BP = scaler_output.inverse_transform(model_BP.predict(x_test))

In [ ]:
def show_data(test_y,predict_y,title):
    plt.figure(figsize=(10,8),dpi=100)
    x = [i for i in range(test_y.shape[0])]
    plt.plot(x, test_y, color='navy',lw = 2, label='Actual data')
    plt.plot(x, predict_y, color='c',lw = 2, label='Predict data')
    x_label = [i for i in x[::5]]
    
    y_label = [j for j in range(0,500,50)]
    plt.yticks(y_label)
    plt.xticks(x_label)
    plt.xlabel('Time/d')
    plt.ylabel('COD')
    plt.title(title)
    plt.legend()
    plt.show()

In [ ]:
show_data(y_test1,y_predict_BP,"GRU")

In [ ]:
data = {"real_cod": [index[0] for index in y_test1.tolist()], "predicted_cod": [index[0] for index in y_predict_BP.tolist()]}
print(data)
frame = pd.DataFrame(data)
frame.to_csv("BP_data.csv", index = False)